In [ ]:
!pip install -q torch torchvision torchaudio
!pip install -q transformers accelerate
!pip install -q peft bitsandbytes
!pip install -q soundfile numpy
!pip install -q elevenlabs
!apt-get install -y libportaudio2 ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.9/682.9 kB 11.2 MB/s eta 0:00:00
Reading package lists... Done
Building depe

In [ ]:
!pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 16.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.4 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803404 sha256=e18536ef18138df54342124deeb7d3dd1d8a5076d922faccd2a73ab197119eee
  Stored in directory: /root/.cache/pip/wheels/2f/f2/ce/6eb23db4091d026238ce76703bd66da60b969d70bcc81d5d3a
Successfully built openai-whisper


In [ ]:
import torch
import numpy as np
import soundfile as sf
import whisper
from IPython.display import Audio, display
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from elevenlabs import ElevenLabs

In [ ]:
class AI_Assistant:
    def __init__(self):
        self.elevenlabs_api_key = "sk_b88b810b8719097ad3455949f2657ecec3e189264673b1a6"  # Replace with your real API key
        self.elevenlabs_client = ElevenLabs(api_key=self.elevenlabs_api_key)

        alif_model_id = "large-traversaal/Alif-1.0-8B-Instruct"

        try:
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4"
            )
            model_kwargs = {"quantization_config": quantization_config}
        except Exception as e:
            print(f"Quantization config error: {e}")
            model_kwargs = {}

        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(alif_model_id, trust_remote_code=True)

        # ✅ Fix: Add chat template manually to avoid tokenizer error
        self.tokenizer.chat_template = (
            "{% for message in messages %}"
            "{% if message['role'] == 'user' %}<|user|>{{ message['content'] }}{% endif %}"
            "{% if message['role'] == 'assistant' %}<|assistant|>{{ message['content'] }}{% endif %}"
            "{% endfor %}<|assistant|>"
        )

        # Try to load PEFT config
        try:
            peft_config = PeftConfig.from_pretrained(alif_model_id)
            base_model_id = peft_config.base_model_name_or_path
        except:
            base_model_id = "mistralai/Mistral-7B-Instruct-v0.2"

        # Load base model
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_id,
            **model_kwargs,
            device_map="auto",
            torch_dtype=torch.float16 if "quantization_config" in model_kwargs else (
                torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float32
            ),
            trust_remote_code=True
        )

        # Load PEFT model on top of base model
        self.model = PeftModel.from_pretrained(base_model, alif_model_id)
        self.model.eval()  # Put model in evaluation mode

        # Load Whisper model for transcription
        self.whisper_model = whisper.load_model("small")

        self.full_transcript = [
            {"role": "system", "content": "You are a helpful and friendly voice assistant. Respond concisely and keep answers brief."}
        ]

    def transcribe_from_file(self, file_path):
        result = self.whisper_model.transcribe(file_path)
        transcribed_text = result["text"].strip()
        print(f"User: {transcribed_text}")
        class MockTranscript:
            def __init__(self, text):
                self.text = text
        return MockTranscript(transcribed_text)

    def generate_ai_response(self, transcript):
        user_message_content = transcript.text
        user_message = {"role": "user", "content": user_message_content}
        self.full_transcript.append(user_message)

        try:
            # Apply chat template
            prompt = self.tokenizer.apply_chat_template(
                self.full_transcript,
                tokenize=False,
                add_generation_prompt=True
            )

            # Tokenize
            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

            # Generate response
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=1000,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.9
                )

            # Decode and post-process
            output_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            ai_response_text = output_text.replace(prompt, "").strip()
            ai_response_text = ai_response_text.split("<|user|>")[0].split("### Instruction:")[0].split("### Response:")[-1].strip()

            print(f"AI Assistant: {ai_response_text}")
            self.full_transcript.append({"role": "assistant", "content": ai_response_text})
            self.generate_audio(ai_response_text)

        except Exception as e:
            print(f"Error in AI generation: {e}")
            self.generate_audio("I encountered an error.")

    def generate_audio(self, text):
        try:
            audio_stream = self.elevenlabs_client.text_to_speech.stream(
                text=text,
                voice_id="pNInz6obpgDQGcFmaJgB",  # Default female voice
                model_id="eleven_multilingual_v2",
                output_format="pcm_16000"
            )
            audio_data = b''.join(chunk for chunk in audio_stream)
            audio_np = np.frombuffer(audio_data, dtype=np.int16)
            sf.write("response.wav", audio_np, 16000)
            display(Audio("response.wav"))
        except Exception as e:
            print(f"Audio generation failed: {e}")
            print("Fallback: " + text)

In [ ]:
# Step 1: Upload your .wav file
from google.colab import files
uploaded = files.upload()
audio_path = next(iter(uploaded))

# Step 2: Run AI Assistant with uploaded audio
ai = AI_Assistant()
transcript = ai.transcribe_from_file(audio_path)
ai.generate_ai_response(transcript)


Saving testing-pause (online-audio-converter.com).wav to testing-pause (online-audio-converter.com).wav


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

100%|███████████████████████████████████████| 461M/461M [00:09<00:00, 50.7MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

User: پاکستان کی دارل حکومت کا نام کیا ہے؟ پاکستان کے بارے میں کوئی سی پاس چیزیں بتائیں
AI Assistant: پاکستان کی دارل حکومت کا نام اسلام آباد ہے۔
